To perform total aggregations we can also leverage `agg` function.

In [0]:
order_items = spark.read.json('/public/retail_db_json/order_items')

In [0]:
order_items.dtypes

Out[2]: [('order_item_id', 'bigint'),
 ('order_item_order_id', 'bigint'),
 ('order_item_product_id', 'bigint'),
 ('order_item_product_price', 'double'),
 ('order_item_quantity', 'bigint'),
 ('order_item_subtotal', 'double')]

* Get revenue using `order_item_subtotal` for a given `order_item_order_id` (eg: 2)

In [0]:
order_items.filter('order_item_order_id = 2')

Out[3]: DataFrame[order_item_id: bigint, order_item_order_id: bigint, order_item_product_id: bigint, order_item_product_price: double, order_item_quantity: bigint, order_item_subtotal: double]

In [0]:
order_items.filter('order_item_order_id = 2').show()

+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+
order_item_id|order_item_order_id|order_item_product_id|order_item_product_price|order_item_quantity|order_item_subtotal|
+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+
 2| 2| 1073| 199.99| 1| 199.99|
 3| 2| 502| 50.0| 5| 250.0|
 4| 2| 403| 129.99| 1| 129.99|
+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+

In [0]:
from pyspark.sql.functions import sum

In [0]:
help(sum)

Help on function sum in module pyspark.sql.functions:

sum(col)
 Aggregate function: returns the sum of all values in the expression.
 
 .. versionadded:: 1.3

In [0]:
help(order_items.agg)

Help on method agg in module pyspark.sql.dataframe:

agg(*exprs) method of pyspark.sql.dataframe.DataFrame instance
 Aggregate on the entire :class:`DataFrame` without groups
 (shorthand for ``df.groupBy().agg()``).
 
 .. versionadded:: 1.3.0
 
 Examples
 --------
 >>> df.agg({"age": "max"}).collect()
 [Row(max(age)=5)]
 >>> from pyspark.sql import functions as F
 >>> df.agg(F.min(df.age)).collect()
 [Row(min(age)=2)]

In [0]:
help(order_items.groupBy('order_item_order_id').agg)

Help on method agg in module pyspark.sql.group:

agg(*exprs) method of pyspark.sql.group.GroupedData instance
 Compute aggregates and returns the result as a :class:`DataFrame`.
 
 The available aggregate functions can be:
 
 1. built-in aggregation functions, such as `avg`, `max`, `min`, `sum`, `count`
 
 2. group aggregate pandas UDFs, created with :func:`pyspark.sql.functions.pandas_udf`
 
 .. note:: There is no partial aggregation with group aggregate UDFs, i.e.,
 a full shuffle is required. Also, all the data of a group will be loaded into
 memory, so the user should be aware of the potential OOM risk if data is skewed
 and certain groups are too large to fit in memory.
 
 .. seealso:: :func:`pyspark.sql.functions.pandas_udf`
 
 If ``exprs`` is a single :class:`dict` mapping from string to string, then the key
 is the column to perform aggregation on, and the value is the aggregate function.
 
 Alternatively, ``exprs`` can also be a list of aggregate :class:`Column` expressions.
 
 .. versionadded:: 1.3.0
 
 Parameters
 ----------
 exprs : dict
 a dict mapping from column name (string) to aggregate functions (string),
 or a list of :class:`Column`.
 
 Notes
 -----
 Built-in aggregation functions and group aggregate pandas UDFs cannot be mixed
 in a single call to this function.
 
 Examples
 --------
 >>> gdf = df.groupBy(df.name)
 >>> sorted(gdf.agg({"*": "count"}).collect())
 [Row(name='Alice', count(1)=1), Row(name='Bob', count(1)=1)]
 
 >>> from pyspark.sql import functions as F
 >>> sorted(gdf.agg(F.min(df.age)).collect())
 [Row(name='Alice', min(age)=2), Row(name='Bob', min(age)=5)]
 
 >>> from pyspark.sql.functions import pandas_udf, PandasUDFType
 >>> @pandas_udf('int', PandasUDFType.GROUPED_AGG) # doctest: +SKIP
 ... def min_udf(v):
 ... return v.min()
 >>> sorted(gdf.agg(min_udf(df.age)).collect()) # doctest: +SKIP
 [Row(name='Alice', min_udf(age)=2), Row(name='Bob', min_udf(age)=5)]

In [0]:
order_items.filter('order_item_order_id = 2').agg(sum('order_item_subtotal')).show()

+------------------------+
sum(order_item_subtotal)|
+------------------------+
 579.98|
+------------------------+

In [0]:
order_items.filter('order_item_order_id = 2').agg(sum('order_item_subtotal').alias('order_revenue')).show()

+-------------+
order_revenue|
+-------------+
 579.98|
+-------------+

* Get number of items, total quantity as well as revenue for a given order item order id (eg: 2)
  * Number of items can be computed using `count` on `order_item_quantity`.
  * Total quantity can be computed using `sum` on `order_item_quantity`.
  * Total Revenue can be computed using `sum` on `order_item_subtotal`.

In [0]:
from pyspark.sql.functions import count

In [0]:
order_items. \
    filter('order_item_order_id = 2'). \
    show()

+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+
order_item_id|order_item_order_id|order_item_product_id|order_item_product_price|order_item_quantity|order_item_subtotal|
+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+
 2| 2| 1073| 199.99| 1| 199.99|
 3| 2| 502| 50.0| 5| 250.0|
 4| 2| 403| 129.99| 1| 129.99|
+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+

In [0]:
order_items. \
    filter('order_item_order_id = 2'). \
    agg(
        count('order_item_quantity').alias('order_item_count'),
        sum('order_item_quantity').alias('order_quantity'),
        sum('order_item_subtotal').alias('order_revenue')
    ). \
    show()

+----------------+--------------+-------------+
order_item_count|order_quantity|order_revenue|
+----------------+--------------+-------------+
 3| 7| 579.98|
+----------------+--------------+-------------+

In [0]:
# We can only perform one aggregation per one column using this approach
order_items. \
    filter('order_item_order_id = 2'). \
    agg(
        {'order_item_quantity': 'count', 'order_item_subtotal': 'sum'}
    ). \
    toDF('order_item_count', 'order_revenue'). \
    show()

+----------------+-------------+
order_item_count|order_revenue|
+----------------+-------------+
 579.98| 3|
+----------------+-------------+